# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [88]:
# Use this cell to write your code for Task 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


data = pd.read_csv('house_sales.csv')
missing_city = data["city"].value_counts()["--"]
missing_city

73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [89]:
# Use this cell to write your code for Task 2

data = pd.read_csv("house_sales.csv")


clean_data = data
#Cleaning city
clean_data["city"].replace("--", "Unknown", inplace=True)

#Cleaning sale_price
clean_data.dropna(subset=['sale_price'], inplace=True)
clean_data = clean_data[clean_data['sale_price'] > 0]

#Cleaning sale_date
clean_data['sale_date'].fillna("2023-01-01", inplace=True)

#Cleaning months_listed
clean_data['months_listed'].fillna(clean_data['months_listed'].mean(), inplace=True)
clean_data['months_listed'] = clean_data['months_listed'].round(1)

#Cleaning bedrooms
clean_data = clean_data[clean_data['bedrooms'] > 0]
clean_data['bedrooms'].fillna(round(clean_data['bedrooms'].mean()), inplace=True)

#Cleaning house_type

clean_data['house_type'].replace({'Det.': 'Detached', 'Semi': 'Semi-detached', 'Terr.': 'Terraced'}, inplace=True)
clean_data['house_type'].fillna(clean_data['house_type'].mode()[0], inplace=True)

#Cleaning area
def toInt(x):
    return float(x[:-6])

clean_data["area"] = clean_data["area"].apply(toInt)

clean_data


,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1
...,...,...,...,...,...,...,...,...
1495,1123892,Riverford,198661,2022-02-17,7.3,5,Detached,432.2
1496,1327295,Poppleton,358304,2020-10-10,6.2,6,Detached,599.8
1497,1058161,Riverford,176612,2022-11-01,6.8,4,Detached,359.1
1498,1822037,Teasdale,197827,2021-04-03,5.4,3,Detached,253.7


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [90]:
# Use this cell to write your code for Task 3
data = pd.read_csv("house_sales.csv")

clean_data['bedrooms'].fillna(round(data['bedrooms'].mean()), inplace=True)

price_by_rooms = data.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).reset_index()
price_by_rooms.columns = ['bedrooms', 'avg_price', 'var_price']

price_by_rooms = price_by_rooms.round(1)

price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [91]:
# Use this cell to write your code for Task 4
from sklearn.linear_model import LinearRegression

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("validation.csv")

train_data = pd.get_dummies(train_data, columns=["city", "house_type"], drop_first=True)
test_data = pd.get_dummies(test_data, columns=["city", "house_type"], drop_first=True)

train_data["sale_date"] = pd.to_datetime(train_data["sale_date"])
test_data["sale_date"] = pd.to_datetime(test_data["sale_date"])

train_data["sale_year"] = train_data["sale_date"].dt.year
train_data["sale_month"] = train_data["sale_date"].dt.month
train_data["sale_day"] = train_data["sale_date"].dt.day


test_data["sale_year"] = test_data["sale_date"].dt.year
test_data["sale_month"] = test_data["sale_date"].dt.month
test_data["sale_day"] = test_data["sale_date"].dt.day

X = train_data.drop(columns=["sale_price", "sale_date"])
y = train_data["sale_price"]

X_test = test_data.drop(columns=["sale_date"])

model = LinearRegression()
model.fit(X, y)

predections = model.predict(X_test)

base_result =pd.DataFrame({"house_id": X_test["house_id"], "price": predections}) 
base_result 

,house_id,price
0,1331375,121709.813918
1,1630115,304232.551410
2,1645745,384418.334267
3,1336775,125014.708890
4,1888274,270695.083194
...,...,...
295,1986255,349092.841750
296,1896276,368686.764539
297,1758223,257195.388196
298,1752010,169083.390810


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [92]:
# Use this cell to write your code for Task 5
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(random_state=34)
model2.fit(X, y)

X_test = test_data.drop(columns=["sale_date"])
pricepredicted = model2.predict(X_test)

compare_result = pd.DataFrame({"house_id": test_data["house_id"], "price": pricepredicted})

print(compare_result)

     house_id      price
0     1331375   79706.91
1     1630115  307728.61
2     1645745  402035.21
3     1336775  104777.08
4     1888274  261235.12
..        ...        ...
295   1986255  361336.13
296   1896276  384239.40
297   1758223  262510.78
298   1752010  176432.64
299   1651404  413221.42

[300 rows x 2 columns]
